# Lielo valodas modeļu novērtēšana uz Krievijas vēstures jautājumu JSON kopas

Šajā notebook tiek salīdzinātas desmit LVM:
- `google/gemma-2-27b-it`
- `google/gemma-2-27b`
- `TildeAI/TildeOpen-30b`
- `meta-llama/llama-3-8b-instruct`
- `mistralai/mistral-7b-instruct`
- `qwen/qwen-2.5-7b-instruct`
- `mistralai/mistral-nemo`
- `openai/gpt-4o-mini`
- `x-ai/grok-4-fast`
- `anthropic/claude-3-haiku`

Modeļi tiek testēti uz vienas datu kopas ar vienas pareizās atbildes izvēles vēstures jautājumiem  
(no JSON faila ar laukiem `question`, `options`, `answer` un `subref`).

---

**Atvērt šo notebook Google Colab:**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LtH2y1aQoyqxD6hLZmDKQ9foSuXKDZ1S?usp=sharing)


In [1]:
# Ievadiet Openrouter API key drošā veidā
from getpass import getpass

OPENROUTER_API_KEY = getpass(
    "Ievadiet OpenRouter API atslēgu (var atstāt tukšu, tad tiks izmantoti tikai LUMII servera modeļi): "
).strip()

OPENROUTER_URL = "https://openrouter.ai/api/v1"

Ievadiet OpenRouter API atslēgu (var atstāt tukšu, tad tiks izmantoti tikai LUMII servera modeļi): ··········


In [2]:
# @title Instalēšana un modeļu konfigurācija
!pip install -q openai

from openai import OpenAI

# LUMII servera modeļu adreses + OpenRouter modeļi
models = {
    # --- LUMII MODELI ---
    "google/gemma-2-27b-it": "http://gauja.ailab.lv:30051/v1",
    "google/gemma-2-27b":    "http://gauja.ailab.lv:30052/v1",
    "TildeAI/TildeOpen-30b": "http://gauja.ailab.lv:30053/v1",

    # --- OpenRouter MODEĻI ---
    "meta-llama/llama-3-8b-instruct": OPENROUTER_URL,
    "qwen/qwen-2.5-7b-instruct": OPENROUTER_URL,
    "mistralai/mixtral-8x7b-instruct": OPENROUTER_URL,
    "mistralai/mistral-nemo": OPENROUTER_URL,
    "anthropic/claude-3-haiku": OPENROUTER_URL,
    "openai/gpt-4o-mini": OPENROUTER_URL,
    "x-ai/grok-4-fast": OPENROUTER_URL,
}

# Ja nav ievadīts OpenRouter API key → izņemam OpenRouter modeļus
if not OPENROUTER_API_KEY:
    models = {
        name: url
        for name, url in models.items()
        if not url.startswith("https://openrouter.ai")
    }
    print("⚠️ OpenRouter API key nav ievadīts — izmantojam tikai LUMII modeļus.")


def make_client(model_name: str) -> OpenAI:
    base = models[model_name]

    # Ja tas ir OpenRouter
    if base.startswith("https://openrouter.ai"):
        return OpenAI(
            base_url=base,
            api_key=OPENROUTER_API_KEY,
            default_headers={
                "HTTP-Referer": "https://github.com/yourname",
                "X-Title": "LLM-evaluation"
            }
        )

    # LUMII — jebkurš key ir derīgs
    return OpenAI(
        base_url=base,
        api_key="any_key",
    )

list(models.keys())


['google/gemma-2-27b-it',
 'google/gemma-2-27b',
 'TildeAI/TildeOpen-30b',
 'meta-llama/llama-3-8b-instruct',
 'qwen/qwen-2.5-7b-instruct',
 'mistralai/mixtral-8x7b-instruct',
 'mistralai/mistral-nemo',
 'anthropic/claude-3-haiku',
 'openai/gpt-4o-mini',
 'x-ai/grok-4-fast']

In [3]:
# @title JSON faila augšupielāde
from google.colab import files
import json

uploaded = files.upload()  # izvēlies savu .json failu (kā tavs piemērs augstāk)

filename = list(uploaded.keys())[0]
print("Ielādēts fails:", filename)

with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

print("Piemēru skaits datu kopā:", len(data))
print("Pirmais ieraksts:")
data[0]


Saving russian_history_unified_state_exam_matching_task_100_lv.json to russian_history_unified_state_exam_matching_task_100_lv.json
Ielādēts fails: russian_history_unified_state_exam_matching_task_100_lv.json
Piemēru skaits datu kopā: 100
Pirmais ieraksts:


{'ref': 'FFBE42',
 'subref': 'FFBE42_A',
 'source_url': 'https://ege.fipi.ru/bank/index.php?proj=068A227D253BA6C04D0C832387FD0D89',
 'question_type': 'Single Choice',
 'question': 'Kurš no šiem faktiem attiecas uz 1327. gada pretordas sacelšanos Tverā?',
 'options': {'1': 'Sāls dumpis Maskavā',
  '2': 'Smoļenskas aplenkums',
  '3': 'Orenburgas aplenkums',
  '4': 'Nikolaja II atteikšanās no troņa',
  '5': 'Hanu sūtņa Čolhana nogalināšana',
  '6': '“Asiņainā svētdiena” Pēterburgā'},
 'answer': '5'}

In [4]:
# @title Palīgfunkcijas formatēšanai un atbildes iegūšanai
import re
from typing import Optional, Tuple, Dict, Any

def format_mc_question(example: Dict[str, Any]) -> str:
    """
    Sagaida ierakstu:
    - 'question': jautājums
    - 'options': dict ar atslēgām "1","2",... un vērtībām (variantu tekstiem)
    - 'answer': pareizās atbildes numurs kā virkne, piem., "3"
    """
    q = example["question"]
    options_dict = example["options"]

    # Sakārtojam variantus pēc skaitļa
    sorted_items = sorted(options_dict.items(), key=lambda kv: int(kv[0]))

    text = q + "\n"
    for key, opt in sorted_items:
        text += f"{key}: {opt}\n"

    # Maksimāli skaidra, cieta instrukcija
    text += (
        "\nATBILDI TIKAI AR VIENU CIPARU (1–6).\n"
        "NEKO CITU NERAKSTI.\n"
        "IZVADĪT DRĪKST TIKAI VIENU SIMBOLU.\n"
        "\n"
        "Atbilde: "
    )

    return text

def ask_model_mc(prompt, client, model_name, return_raw=False):
    import re

    if model_name.startswith("openrouter/"):
        # OpenRouter → vienmēr chat
        model_id = model_name.replace("openrouter/", "")
        resp = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=4,
        )
        text = resp.choices[0].message.content

    elif model_name.startswith("google/gemma") and model_name.endswith("-it"):
        # gemma IT → chat
        resp = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=4,
        )
        text = resp.choices[0].message.content

    else:
        # citi → completions
        resp = client.completions.create(
            model=model_name,
            prompt=prompt,
            temperature=0,
            max_tokens=4,
        )
        text = resp.choices[0].text

    raw = text.strip()
    m = re.search(r"[1-6]", raw)
    pred = int(m.group(0)) if m else None

    if return_raw:
        return pred, raw
    return pred


In [5]:
# @title Vienas modeļa novērtēšana uz visiem jautājumiem
def evaluate_model(
    model_name: str,
    data,
    verbose: bool = False,
):
    client = make_client(model_name)

    total = 0          # jautājumu kopskaits
    correct = 0        # pareizās atbildes
    skipped = 0        # neizdevās nolasīt (parse) atbildi

    # Papildus metrikam
    y_true = []        # pareizās atbildes (gold)
    y_pred = []        # paredzētas atbildes no LVM (tikai tur, kur nav skipped)

    for ex in data:
        prompt = format_mc_question(ex)

        gold_num = int(ex["answer"])
        ref = ex.get("subref") or ex.get("ref", f"Q{total+1}")

        if verbose:
            print(f"\n=== {model_name} | Subref: {ref} ===")
            print(prompt)

        pred_num, raw_output = ask_model_mc(prompt, client, model_name, return_raw=True)

        total += 1

        if verbose:
            print("\nRAW OUTPUT:", repr(raw_output))

        if pred_num is None:
            skipped += 1
            status = "SKIPPED"
        else:
            # metriku aprēķinā izmantojam tikai atbildes, kuras izdevās nolasīt
            y_true.append(gold_num)
            y_pred.append(pred_num)

            if pred_num == gold_num:
                correct += 1
                status = "OK"
            else:
                status = "WRONG"

        if verbose:
            print(f"Gold: {gold_num} | Pred: {pred_num} → {status}")
            print("-" * 60)

    accuracy = correct / total if total > 0 else 0.0
    evaluated = len(y_true)  # cik daudz piemēru reāli iekļuva metrikās (bez skipped)

    # === Papildus metrikas: confusion matrix + macro P/R/F1 ===
    # ja visas atbildes bija skipped, tad metrikas būs nulles
    if evaluated > 0:
        labels = sorted(set(y_true) | set(y_pred))  # parasti [1,2,3,4,5,6]
        idx = {lab: i for i, lab in enumerate(labels)}
        K = len(labels)

        # confusion matrix: rows = gold, cols = pred
        cm = [[0] * K for _ in range(K)]
        for g, p in zip(y_true, y_pred):
            cm[idx[g]][idx[p]] += 1

        precisions = []
        recalls = []
        f1s = []

        for i, lab in enumerate(labels):
            TP = cm[i][i]
            FP = sum(cm[r][i] for r in range(K) if r != i)
            FN = sum(cm[i][c] for c in range(K) if c != i)

            prec = TP / (TP + FP) if (TP + FP) > 0 else 0.0
            rec = TP / (TP + FN) if (TP + FN) > 0 else 0.0
            f1 = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0

            precisions.append(prec)
            recalls.append(rec)
            f1s.append(f1)

        macro_precision = sum(precisions) / K
        macro_recall = sum(recalls) / K
        macro_f1 = sum(f1s) / K
    else:
        macro_precision = 0.0
        macro_recall = 0.0
        macro_f1 = 0.0

    print(f"\nKopsavilkums modelim: {model_name}")
    print("--------------------------------")
    print("Kopējais piemēru skaits:", total)
    print("Pareizas atbildes:      ", correct)
    print("Izlaisti (parse kļūda): ", skipped)
    print("Accuracy:               ", round(accuracy, 3))
    print("Macro precision:        ", round(macro_precision, 3))
    print("Macro recall:           ", round(macro_recall, 3))
    print("Macro F1:               ", round(macro_f1, 3))

    return {
        "model": model_name,
        "total": total,
        "evaluated": evaluated,           # bez skipped
        "correct": correct,
        "skipped": skipped,
        "accuracy": accuracy,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
    }


In [6]:
# @title Salīdzinājums starp visiem modeļiem
import pandas as pd

results = []

# maini uz False, ja ne gribi pilno izdruku visiem jautājumiem
VERBOSE = True

for model_name in models.keys():
    res = evaluate_model(model_name, data, verbose=VERBOSE)
    results.append(res)


Straumēto izvades datu attēlojums tika saīsināts līdz pēdējām 5000 rindām.
6: Ivans Kaļita

ATBILDI TIKAI AR VIENU CIPARU (1–6).
NEKO CITU NERAKSTI.
IZVADĪT DRĪKST TIKAI VIENU SIMBOLU.

Atbilde: 

RAW OUTPUT: '3'
Gold: 3 | Pred: 3 → OK
------------------------------------------------------------

=== anthropic/claude-3-haiku | Subref: F20D05_C ===
Kurš bija iesaistīts Maskavas kņazistes iekšējā karā 15. gadsimta otrajā ceturtdaļā?
1: Vasilijs II Tumšais
2: M. M. Ļitvinovs
3: D. M. Požarskis
4: E. F. Kankrins
5: S. J. Vite
6: Ivans Kaļita

ATBILDI TIKAI AR VIENU CIPARU (1–6).
NEKO CITU NERAKSTI.
IZVADĪT DRĪKST TIKAI VIENU SIMBOLU.

Atbilde: 

RAW OUTPUT: '1'
Gold: 1 | Pred: 1 → OK
------------------------------------------------------------

=== anthropic/claude-3-haiku | Subref: F20D05_D ===
Kurš ieviesa zelta standartu Krievijas rublim?
1: Vasilijs II Tumšais
2: M. M. Ļitvinovs
3: D. M. Požarskis
4: E. F. Kankrins
5: S. J. Vite
6: Ivans Kaļita

ATBILDI TIKAI AR VIENU CIPARU (1–6).
NEK

In [7]:
df = pd.DataFrame(results)
df

,model,total,evaluated,correct,skipped,accuracy,macro_precision,macro_recall,macro_f1
0,google/gemma-2-27b-it,100,100,74,0,0.74,0.742633,0.729567,0.728763
1,google/gemma-2-27b,100,100,62,0,0.62,0.687160,0.625520,0.617506
2,TildeAI/TildeOpen-30b,100,100,79,0,0.79,0.806283,0.780002,0.776879
3,meta-llama/llama-3-8b-instruct,100,100,32,0,0.32,0.370750,0.318492,0.319767
4,qwen/qwen-2.5-7b-instruct,100,100,39,0,0.39,0.407485,0.373910,0.368222
5,mistralai/mixtral-8x7b-instruct,100,100,37,0,0.37,0.435703,0.375147,0.381142
6,mistralai/mistral-nemo,100,100,48,0,0.48,0.540536,0.479876,0.471075
7,anthropic/claude-3-haiku,100,100,87,0,0.87,0.878901,0.862692,0.868080
8,openai/gpt-4o-mini,100,100,87,0,0.87,0.868353,0.863304,0.863294
9,x-ai/grok-4-fast,100,100,97,0,0.97,0.973091,0.973091,0.973091


# Rezultātu analīze

#### **1) Augstākā grupa (1.00-0.80)**

Labāko sniegumu uzrāda trīs modeļi:  
*   x-ai/grok-4-fast (0.97)
*   anthropic/claude-3-haiku (0.87)
*   openai/gpt-4o-mini (0.87)

Šie modeļi demonstrē gan augstu precizitāti, gan stabilus makro rādītājus (precision/recall/F1), kas nozīmē vienmērīgi zemu kļūdu līmeni visās sešās atbilžu klasēs.

**GPT-4o-mini** ir optimālākais risinājums no cenas–veiktspējas viedokļa ($0.15/M input tokens).

**Grok-4-Fast** ir nedaudz dārgāks ($0.20/M input tokens), taču sasniedz gandrīz perfektu rezultātu.

**Claude-3-Haiku** ir līdzīgs GPT-4o-mini precizitātes ziņā, bet ir dārgāks ($0.25/M input tokens).

<br>

---

<br>

#### **2) Vidējā grupa (0.80-0.60)**

Šajā grupā ietilpst:

- TildeAI/TildeOpen-30B (0.79)
- google/gemma-2-27b-it (0.74)
- google/gemma-2-27b (0.62)

**TildeOpen-30B** ir labākais atvērtā koda modelis šajā testā un vistuvāk komerciālajiem modeļiem.

Tas īpaši labi tiek galā ar sarežģītiem, no krievu valodas tulkotiem vēstures terminiem, kur citi modeļi biežāk kļūdās.

**Gemma-2-27b-it** snieguma ziņā ir tuvs TildeOpen-30B

**Gemma-2-27b** kļūdās biežāk, īpaši mazizplatītos vēsturiskos terminos.

<br>

---

<br>

#### **3) Zemākā grupa (mazie modeļi) (0.60-0.00)**

Šajā grupā ietilpst:

- mistral-nemo (0.48)
- qwen-2.5-7b (0.39)
- mixtral-8x7b (0.37)
- llama-3-8b (0.32)

Tie bieži kļūdās datumos, personu identificēšanā un notikumu klasifikācijā, tāpēc tie nav piemēroti faktoloģiskiem Krievijas vēstures uzdevumiem. Šie modeļi nav ieteicami šāda tipa testiem.

<br>

---